# Quick tests outside of Test macro

In [1]:
using GalacticDynamics
using Test
using PythonCall
using DifferentialEquations
using CairoMakie
using BenchmarkTools


pyimport("sys")."path".append("")
pyimport("sys")."path".append("python")
accelerations_py = pyimport("accelerations")
au = pyimport("astropy.units")
gd = pyimport("gala.dynamics")
gp = pyimport("gala.potential")
gu = pyimport("gala.units")
gi = pyimport("gala.integrate")

Python: <module 'gala.integrate' from '/home/mmestre/.conda/envs/cosmos/lib/python3.11/site-packages/gala/integrate/__init__.py'>

In [41]:

t₁, t₂ = 0.0, 3.0
t_range = (t₁, t₂)
Δt = 0.01
n_max = (t₂-t₁)/Δt
x₀ = -50.0*[1,0,0]
v₀ = 200.0*[0,1,0]
m = 10.0^12*𝕦.m  # Msun
a = 20.0*𝕦.l
pot = NFW(m, a)
c = concentration(pot)
f(x) = log(1+x)-x/(1+x)
const 𝔹 = f(c)
@show 𝔹
m_g = m/f(c)
usys = gu.UnitSystem(au.kpc, au.Gyr, au.Msun, au.radian, au.kpc/au.Gyr, au.kpc/au.Gyr^2)
pot_Gala = gp.NFWPotential(Py(adimensional(m_g))*au.Msun, Py(adimensional(a))*au.kpc, units=usys)
w₀ = gd.PhaseSpacePosition(pos=Py(x₀)*au.kpc, vel=Py(v₀)*au.kpc/au.Gyr)

𝔹 = 1.533297373418205


Python: <PhaseSpacePosition cartesian, dim=3, shape=()>

In [42]:
acc =  acceleration(pot,x₀)[1]
acc_Gala = pyconvert(Vector{Float64},pot_Gala.acceleration(w₀)[0])[1];
acc_diff = abs(acc - acc_Gala)

0.0

In [45]:
sol_gala = pot_Gala.integrate_orbit(w₀, dt=Δt*au.Gyr, t1=t₁, t2=(t₂)*au.Gyr,
                                    Integrator=gi.DOPRI853Integrator,
                                    Integrator_kwargs=Dict("rtol"=>5.0e-12, "atol"=>0.5e-12))
sol = evolve(pot, x₀, v₀, t_range; options=ntSolverOptions(; reltol=5.0e-12, abstol=0.5e-12,saveat=Δt))
@show 𝕤.ode
@show sol_gala.t[-1]
@show sol.t[end]
@show sol_gala.x[0+100]
@show sol.x[1,:][1+100]
orb_gala_x = pyconvert(Vector{Float64}, sol_gala.x)
orb_gala_y = pyconvert(Vector{Float64}, sol_gala.y)
orb_x = sol.x[1,:]
orb_y = sol.x[2,:]
sol_diff = abs.(orb_gala_x-orb_x)
@show findmax(sol_diff)
lines(orb_gala_x,orb_gala_y)
lines!(orb_x,orb_y)
current_figure()



MethodError: MethodError: no method matching evolve(::NFW{Float64, Float64}, ::Vector{Float64}, ::Vector{Float64}, ::Tuple{Float64, Float64}; options::@NamedTuple{abstol::Float64, reltol::Float64, saveat::Float64})

Closest candidates are:
  evolve(::P, !Matched::TestParticle, !Matched::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}, ::Any; options) where P<:Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}
   @ GalacticDynamics ~/.julia/dev/GalacticDynamics/src/evolutions.jl:33
  evolve(::P, !Matched::Event, !Matched::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}, ::Any; options) where P<:Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}
   @ GalacticDynamics 

In [46]:
methods(evolve)

# 8 methods for generic function "evolve" from GalacticDynamics:
 [1] evolve(mps::Vector{P}, t_span::Tuple{T, T}, solver; options) where {P<:GalacticDynamics.AbstractMacroParticle, T<:Real}
     @ ~/.julia/dev/GalacticDynamics/src/evolutions.jl:42
 [2] evolve(mps::Vector{P}, t_span::Tuple{T, T}; ...) where {P<:GalacticDynamics.AbstractMacroParticle, T<:Real}
     @ ~/.julia/dev/GalacticDynamics/src/evolutions.jl:42
 [3] evolve(pot::P, p::TestParticle, t_span::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}, solver; options) where P<:Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}
     @ ~/.julia/dev/GalacticDynamics/src/evolutions.jl:33
 [4] evolve(pot::Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}, x::Vector{<:Union{Quantity{T, 𝐋, U}, Level{L, S, Quantity{T, 𝐋, U}} where {L, S}} where {T, U}}, v::Vector{<:Union{Quantity{T, 𝐋 𝐓^-1, U}, Level{L, S, Quantity{T, 𝐋 𝐓^-1, U}} where {L, S}} where {T, U}}, t_span::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}; ...)
     @ ~/.julia/dev/GalacticDynamics/src/evolutions.jl:16
 [5] evolve(pot::Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}, x::Vector{<:Union{Quantity{T, 𝐋, U}, Level{L, S, Quantity{T, 𝐋, U}} where {L, S}} where {T, U}}, v::Vector{<:Union{Quantity{T, 𝐋 𝐓^-1, U}, Level{L, S, Quantity{T, 𝐋 𝐓^-1, U}} where {L, S}} where {T, U}}, t_span::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}, solver; options)
     @ ~/.julia/dev/GalacticDynamics/src/evolutions.jl:16
 [6] evolve(pot::P, event::Event, t_span::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}; ...) where P<:Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}
     @ ~/.julia/dev/GalacticDynamics/src/evolutions.jl:24
 [7] evolve(pot::P, event::Event, t_span::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}, solver; options) where P<:Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}
     @ ~/.julia/dev/GalacticDynamics/src/evolutions.jl:24
 [8] evolve(pot::P, p::TestParticle, t_span::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}; ...) where P<:Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}
     @ ~/.julia/dev/GalacticDynamics/src/evolutions.jl:33

In [31]:
@benchmark pot_Gala.integrate_orbit(w₀, dt=Δt*au.Gyr, t1=t₁, t2=(t₂)*au.Gyr,
                                    Integrator=gi.DOPRI853Integrator,
                                    Integrator_kwargs=Dict("rtol"=>5.0e-8))


BenchmarkTools.Trial: 2889 samples with 1 evaluation.
 Range (min … max):  1.431 ms …  45.189 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.632 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.717 ms ± 849.900 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▇█▆▂                                                        
  ▃█████▆▅▅▄▄▄▅▅▆▆▆▇▆▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.43 ms         Histogram: frequency by time        2.69 ms <

 Memory estimate: 2.27 KiB, allocs estimate: 89.

In [40]:
evolve(pot, x₀, v₀, t_range, Vern7(); options=ntSolverOptions(; reltol=5.0e-10, saveat=Δt))

MethodError: MethodError: no method matching evolve(::NFW{Float64, Float64}, ::Vector{Int64}, ::Vector{Int64}, ::Tuple{Float64, Float64}, ::Vern7{typeof(OrdinaryDiffEq.trivial_limiter!), typeof(OrdinaryDiffEq.trivial_limiter!), Static.False}; options::@NamedTuple{abstol::Float64, reltol::Float64, saveat::Float64})

Closest candidates are:
  evolve(::P, !Matched::TestParticle, !Matched::Tuple{Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}, Union{Quantity{T, 𝐓, U}, Level{L, S, Quantity{T, 𝐓, U}} where {L, S}} where {T, U}}, ::Any; options) where P<:Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}
   @ GalacticDynamics ~/.julia/dev/GalacticDynamics/src/evolutions.jl:33
  evolve(::Union{GalacticDynamics.AbstractPotential, Vector{<:GalacticDynamics.AbstractPotential}}, !Matched::Vector{<:Union{Quantity{T, 𝐋, U}, Level{L, S, Quantity{T, 𝐋, U}} where {L, S}} where {T, U}}, !Matched::Vector{<:Union{Quantity{T, 𝐋 𝐓^-1, U